# ActiveRecord

Zanim przystąpimy do pracy, musimy skonfigurować bazę danych. Ponieważ pracujemy bez użycia Railsów, konieczne jest
ręczne zestawienie połączenia oraz stworzenie odpowiednich tabel w bazie. Zadania te realizowane są przez
skrypt `db_setup.rb`. Aby go uruchomić wpisujemy
```ruby
$:.unshift "."
require 'db_setup'
```

In [1]:
$:.unshift "."
require 'db_setup'


==  CreateAuthorsMigration: migrating =========================================
-- create_table(:authors)
   -> 0.1621s
==  CreateAuthorsMigration: migrated (0.1640s) ================================

==  CreateBooksMigration: migrating ===========================================
-- create_table(:books)
   -> 0.1367s
==  CreateBooksMigration: migrated (0.1396s) ==================================

==  CreateGenresMigration: migrating ==========================================
-- create_table(:genres)
   -> 0.1284s
==  CreateGenresMigration: migrated (0.1299s) =================================

==  CreateBooksGenresMigration: migrating =====================================
-- create_table(:books_genres, {:id=>false})
   -> 0.1312s
-- add_index(:books_genres, [:book_id, :genre_id], {:unique=>true})
   -> 0.1760s
==  CreateBooksGenresMigration: migrated (0.3116s) ============================

==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigr

true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # language  (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
  has_and_belongs_to_many :genres
end

class Genre < ActiveRecord::Base
  # name  (string)
  
  has_and_belongs_to_many :books
end
```

## CRUD

Cztery podstawowe operacje, które wykonujemy na danych to
* tworzenie - **C**reate
* odczytywanie - **R**ead
* modyfikowanie - **U**pdate
* usuwanie - **D**elete

W skrócie oznaczane są one za pomocą akronimu CRUD.

### Create

W ActiveRecord (w skrócie AR) korzystamy z obiektowego interfejsu. 
Tworzenie danych wygląda następująco:
```ruby
author = Author.new(name: "Adam", surname: "Mickiewicz")
author.save
```

In [2]:
author = Author.new(name: "Adam", surname: "Mickiewicz")
author.save


true

Innymi słowy tworzymy nowy obiekt Rubiego i wywołujemy na nim metodę `save`. To że faktycznie został on dodany do bazy danych 
możemy zweryfikować wyszukując pierwszy obiekt w bazie:
```ruby
author = Author.first
puts author.name
puts author.surname
```

In [3]:
author = Author.first
puts author.name
puts author.surname


Adam
Mickiewicz


### Zadanie 1

Dodaj do bazy 3 autorów:
* Juliusz Słowacki
* Henryk Sienkiewicz
* Eliza Orzeszkowa

In [10]:
author = Author.new(name: "Juliusz", surname: "Słowacki")
author.save
author = Author.new(name: "Henryk", surname: "Sienkiewicz")
author.save
author = Author.new(name: "Eliza", surname: "Orzeszkowa")
author.save


true

### Read

Odczytywanie danych z bazy można realizować na wiele sposobów. Najprostszy sposób, to wyszukiwanie ich z wykorzystaniem 
klucza główego - `id`. Służy do tego metoda `find`:
```ruby
author = Author.find(1)
puts author.surname
author = Author.find(2)
puts author.surname
```

In [22]:
author = Author.find(2)
puts author.surname
author = Author.find(4)
puts author.surname
author = Author.find(5)
puts author.surname
author = Author.find(6)
puts author.surname


Mickiewicz
Słowacki
Sienkiewicz
Orzeszkowa


Wykorzystanie tej metody może jedak skutkować wyjątkiem, jeśli w bazie nie ma wiersza z danym kluczem:
```ruby
author = Author.find(10)
```

In [183]:
author = Author.find(10)

ActiveRecord::RecordNotFound: Couldn't find Author with 'id'=10

Możemy zabezpieczyć się przed tą sytuacją, korzystają z innego wywołania `find_by_id`
```ruby
author = Author.find_by_id(10)
p author
```

In [185]:
author = Author.find_by_id(10)
p author


nil


### Zadanie 2

Próba odczytania pól takiego obiektu, również skończy się wyjątkiem. Co należy zrobić, żeby wypisać imię i nazwisko autora
wyłącznie wtedy gdy autor istnieje w bazie? Zaimplementuj metodę `print_author`, która radzi sobie z tym problemem.

In [186]:
def print_author(id)
  author = Author.find_by_id(id)
  p author
end

# te linijki mają pozostać niezmienione
print_author(1)
print_author(10)

#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>
nil


### Update

Modyfikowanie danych realizowane może być na kilka sposobów. W pierwszej kolejności możemy zmodyfikować atrybut obiektu i 
następnie zapisać go do bazy
```ruby
author = Author.find(1)
puts author.surname
author.surname = "Mickiewiczowski"
author.save

other_author = Author.find(1)
puts other_author.surname
```

In [33]:
author = Author.find(1)
puts author.surname
author.surname = "Mickiewiczowski"
author.save

other_author = Author.find(1)
puts other_author.surname


Mickiewicz
Mickiewiczowski


Można również skorzystać z metody `update_attributes`, która działa podobnie jak konstruktor, ale dane są od razu
modyfikowane w bazie
```ruby
author = Author.find(1)
author.update_attributes(name: "Wojciech")
other_author = Author.find(1)
puts other_author.name
```

### Zadanie 3

Zmodyfikuj wszystkich autorów, tak by ich daty urodzenia i śmierci były poprawne. Popraw również imię i nazwisko Adama Mickiewicza.
Aby wprowadzić datę skorzystaj z metody `Date.parse`.

In [125]:
author = Author.find(4)
author.update_attributes(born: Date.parse('June 6, 1809').to_s)
author.update_attributes(died: Date.parse('November 18, 1849').to_s)
other_author = Author.find(4)
p other_author

p Author.find(4)



#<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>
#<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>


#<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>

### Delete

Usuwanie danych realizowane jest za pomocą wywołania `destroy`:
```ruby
author = Author.find(1)
author.destroy
```

In [180]:
author = Author.find(2)
author.destroy

#<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>

### Zadanie 4

Ponieważ właśnie usunąłeś/ęłaś Adama Mickiewicza, ponownie utwórz odpowiadający mu rekord.

In [158]:
author = Author.new(id: 2, name: "Adam", surname: "Mickiewicz", born: Date.parse('December 24, 1772').to_s, died: Date.parse('November 26, 1855').to_s) 
author.save


ActiveRecord::RecordNotUnique: SQLite3::ConstraintException: UNIQUE constraint failed: authors.id: INSERT INTO "authors" ("id", "name", "surname", "born", "died") VALUES (?, ?, ?, ?, ?)

## Język zapytań

## `find`, `first`, `last`, `all`

Metoda `find` pozwala nie tylko pobierać pojedynczy obiekt z bazy, ale również kilka obiektów na raz:
```ruby
authors = Author.find(2,3,4)
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end
```

In [138]:
authors = Author.find(2,3,4)
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end


Adam Mickiewicz
Eliza Orzeszkowa
Juliusz Słowacki


[#<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>]

Metody `first` oraz `last` zwracają odpowiednio *pierwszy* i *ostatni* rekord w bazie. W domyślnej konfiguracji kolejność
ta będzie odpowiadała czasowi ich utworzenia.
```ruby
puts Author.first.surname
puts Author.last.surname
```

In [139]:
puts Author.first.surname
puts Author.last.surname


Mickiewicz
Sienkiewicz


Natomiast metoda `all` zwraca kolekcję obejmującą wszystkie rekordy w bazie danych:
```ruby
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end
```

In [140]:
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end


Adam Mickiewicz
Adam Mickiewicz
Eliza Orzeszkowa
Juliusz Słowacki
Henryk Sienkiewicz


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>]

### Zadanie 5

Wypisz wszystkich autorów znajdujących się w bazie wraz z ich datami urodzenia i śmierci. Postaraj się sformatować daty,
tak by obejmowały tylko dzień, miesiąc i rok - w tej kolejności. Służy do tego metoda `strftime`.

In [147]:
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname}     born: #{author.born.strftime("%d-%m-%Y")}     died: #{author.died.strftime("%d-%m-%Y")}"
end

Adam Mickiewicz     born: 24-12-1798     died: 26-11-1855
Adam Mickiewicz     born: 24-12-1772     died: 26-11-1855
Eliza Orzeszkowa     born: 06-06-1841     died: 18-11-1910
Juliusz Słowacki     born: 06-06-1809     died: 18-11-1849
Henryk Sienkiewicz     born: 06-06-1846     died: 18-11-1916


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>]

### `find_by`

AR definiuje również metody pozwalające na wyszukiwanie rekordów na podstawie wartości atrybutów. Najprostsza z nich to `find_by`. Zwraca rekord, które posiada wartość określoną w zapytaniu:
```ruby
author = Author.find_by_name("Adam")
puts author.surname
```

In [148]:
author = Author.find_by_name("Adam")
puts author.surname


Mickiewicz


### `where`

Metoda `where` odpowiada klauzuli `where` z języka SQL. Podstawowa różnica polega na tym, że wartości poszczególnych pól określamy w postaci par klucz-wartość. Jeśli chcemy uzyskać pojedynczy wynik dodajemy metodę `first` lub `last`:
```ruby
author = Author.where(name: "Eliza").first
puts author.surname
```

In [151]:
author = Author.where(name: "Eliza").first
puts author.surname


Orzeszkowa


Metoda ta ma jednak znacznie większe możliwości - można np. podawać zakresy wartości, jako zakresy Rubiego:
```ruby
authors = Author.where(born: (Date.parse("1780-1-1")..Date.parse("1800-12-31"))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end
```

In [152]:
authors = Author.where(born: (Date.parse("1780-1-1")..Date.parse("1800-12-31")))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end


Adam Mickiewicz 24-12-1798


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>]

Metodę `where` można wywoływać wielokrotnie. Wtedy wyniki są łączone za pomocą operatora koniunkcji. Jeśli chcemy
użyć innego operatora (np. `OR` lub `LIKE`), konieczne jest użycie nieco innej składni:
```ruby
authors = Author.where("name LIKE 'A%'")
authors.each do |author|
  puts author.surname
end
```  

In [153]:
authors = Author.where("name LIKE 'A%'")
authors.each do |author|
  puts author.surname
end


Mickiewicz
Mickiewicz


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>]

Jeśli dane w napisie przekazanym do metody `where` pochodzą od użytkownika aplikacji, to narażamy się na atak SQL-injection.
Aby go unikąć, wartość podaną przez użytkownika przekazujemy jako osobny argument, np.
```ruby
name = "Adam"
authors = Author.where("name = ?",name)
authors.each do |author|
  puts author.surname
end
```

In [154]:
name = "Adam"
authors = Author.where("name = ?",name)
authors.each do |author|
  puts author.surname
end


Mickiewicz
Mickiewicz


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>]

W tej sytuacji AR sam zadba o odpowiednią konwersję znaków "niebezpiecznych".

### Zadanie 6

Znajdź i wypisz wszystkich autorów, którzy zmarli między rokiem 1800 a 1900.

In [166]:
authors = Author.where(died: (Date.parse("1800-1-1")..Date.parse("1900-12-31")))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end

Adam Mickiewicz 24-12-1798
Adam Mickiewicz 24-12-1772
Juliusz Słowacki 06-06-1809


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>]

### `order`

Do określania kolejności wyników służy metoda `order`. Działa ona analogicznie do klauzuli `ORDER` w języku SQL.
```ruby
authors = Author.order(:born)
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [167]:
authors = Author.order(:born)
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end


Adam Mickiewicz 1772-12-24
Adam Mickiewicz 1798-12-24
Juliusz Słowacki 1809-06-06
Eliza Orzeszkowa 1841-06-06
Henryk Sienkiewicz 1846-06-06


[#<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>]

Metoda ta często jest łączona z wywołaniami `first` i `last`
```ruby
author = Author.order(:born).last
puts "#{author.name} #{author.surname} #{author.born}"
```

### Zadanie 7

Znajdź autora, który zmarł jako ostatni.

In [170]:
author = Author.order(:died).last
puts "#{author.name} #{author.surname} #{author.died}"


Henryk Sienkiewicz 1916-11-18


### `limit` i `offset`

Metody `limit` i `offset` działają analogicznie jak ich odpowiedniki w SQL:
```ruby
Author.limit(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [171]:
Author.limit(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end


Adam Mickiewicz 1798-12-24
Adam Mickiewicz 1772-12-24


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Adam", surname: "Mickiewicz", born: "1772-12-24", died: "1855-11-26", image_url: nil>]

```ruby
Author.offset(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [172]:
Author.offset(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end



Eliza Orzeszkowa 1841-06-06
Juliusz Słowacki 1809-06-06
Henryk Sienkiewicz 1846-06-06


[#<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>, #<Author id: 5, name: "Henryk", surname: "Sienkiewicz", born: "1846-06-06", died: "1916-11-18", image_url: nil>]

Warto jednak pamiętać, żeby stosując je określić pożądek rekordów.

### Zadanie 8

Znajdź i wypisz 3 autorów, którzy zmarli jako pierwsi.

In [181]:
Author.limit(3).each do |author|
  puts "#{author.name} #{author.surname} #{author.died}"
end

Adam Mickiewicz 1855-11-26
Eliza Orzeszkowa 1910-11-18
Juliusz Słowacki 1849-11-18


[#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 3, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-11-18", image_url: nil>, #<Author id: 4, name: "Juliusz", surname: "Słowacki", born: "1809-06-06", died: "1849-11-18", image_url: nil>]